In [32]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import seaborn as sns

In [33]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
from pandas.errors import SettingWithCopyWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))
warnings.simplefilter(action='ignore', category=(FutureWarning))

In [34]:
defense = pd.read_csv("../match_logs/Big5@22-23@defense.csv")
gca = pd.read_csv("../match_logs/Big5@22-23@gca.csv")
keeper = pd.read_csv("../match_logs/Big5@22-23@keeper.csv")
misc = pd.read_csv("../match_logs/Big5@22-23@misc.csv")
passing = pd.read_csv("../match_logs/Big5@22-23@passing.csv")
passing_types = pd.read_csv("../match_logs/Big5@22-23@passing_types.csv")
possession = pd.read_csv("../match_logs/Big5@22-23@possession.csv")
shooting = pd.read_csv("../match_logs/Big5@22-23@shooting.csv")

In [35]:
team_ratings = pd.read_csv("../players_db/fm23/team_ratings.csv")

In [36]:
team_ratings_cols = team_ratings.columns

In [37]:
tpr_cols = []
except_cols = ["Club","Based","Division","Club_id","League_id","fbref_name"]
for col in team_ratings_cols:
    if col not in except_cols:
        for xy in ["x","y"]:
            xy_col = f"{col}_{xy}"
            tpr_cols.append(xy_col)

<br><br><br><br><br><br><br><br>
<h1 style="color:orange;background:blue;">@.  Logs to Regression</h1>

In [52]:
CURRENT_DATAFRAME_NAME = "passing"
CURRENT_DATAFRAME = locals()[CURRENT_DATAFRAME_NAME]
CURRENT_DATAFRAME["Unnamed: 0"] = "itsanobject"

In [53]:
# CURRENT_DATAFRAME["Unique_Matchday_id"] = ((CURRENT_DATAFRAME.Club).apply(hash)+(CURRENT_DATAFRAME.Opponent).apply(hash)).apply(str)
# unique_indexes = CURRENT_DATAFRAME["Unique_Matchday_id"].drop_duplicates().index
# CURRENT_DATAFRAME = CURRENT_DATAFRAME.loc[unique_indexes]

In [54]:
DATAFRAME_COLUMNS = list(CURRENT_DATAFRAME.columns)
NUMERIC_DATAFRAME_COLUMNS = list(CURRENT_DATAFRAME.select_dtypes(exclude="object").columns)

In [73]:
CURRENT_DATAFRAME[NUMERIC_DATAFRAME_COLUMNS].describe().loc[['count','min', 'mean', 'max']].T[["mean"]].apply(round)

,mean
GF,1.0
GA,1.0
Cmp,385.0
Att,487.0
Cmp%,78.0
TotDist,6880.0
PrgDist,2508.0
Cmp.1,171.0
Att.1,196.0
Cmp%.1,86.0


In [56]:
merged_df_1 = pd.merge(CURRENT_DATAFRAME, team_ratings, on="Club", how="inner")  
merged_df = pd.merge(merged_df_1, team_ratings, left_on="Opponent", right_on="fbref_name", how="inner")  

In [57]:
MERGED_NUMERIC_DATAFRAME_COLUMNS = list(merged_df.select_dtypes(exclude="object").columns)

In [58]:
reg_df = merged_df[['Club_x','Club_y'] + MERGED_NUMERIC_DATAFRAME_COLUMNS] 
reg_df.iloc[0]

Club_x              Manchester City
Club_y              West Ham United
GF                              2.0
GA                              0.0
Cmp                           792.0
                         ...       
Marking_y                      70.0
Passing_y                      71.0
Penalty Taking_y               64.0
Tackling_y                     77.0
Technique_y                    77.0
Name: 0, Length: 106, dtype: object

<br><br><br><br><br><br><br><br>
<h3 style="color:white;background:red;">  Regression</h3>

In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

In [67]:
formula_df = pd.DataFrame(index=tpr_cols)
np.random.seed(0)
df = reg_df.dropna(axis=0)

scaler = StandardScaler()
df[tpr_cols] = scaler.fit_transform(df[tpr_cols])

for attribute in NUMERIC_DATAFRAME_COLUMNS:
    X = df[tpr_cols]
    Y = df[[attribute]]
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
    model = LinearRegression()
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    
    coef_column_name = f"{attribute}_coef"
    intercept_column_name = f"{attribute}_intercept"
    formula_df[coef_column_name] = model.coef_.flatten()
    formula_df[intercept_column_name] = model.intercept_[0]

    mse = mean_squared_error(Y_test, Y_pred)
    r2 = r2_score(Y_test, Y_pred)
    # print(f'{attribute} --- Mean Squared Error: {mse}')
    # print(f'{attribute} --- R^2 Score: {r2}')
    # print("\n")

In [66]:
formula_df

,GF_coef,GF_intercept,GA_coef,GA_intercept,Cmp_coef,Cmp_intercept,Att_coef,Att_intercept,Cmp%_coef,Cmp%_intercept,...,KP_coef,KP_intercept,1/3_coef,1/3_intercept,PPA_coef,PPA_intercept,CrsPA_coef,CrsPA_intercept,PrgP_coef,PrgP_intercept
GK_x,0.004524,1.953064,-0.009099,-0.161194,0.525624,-320.834053,0.505079,-127.413961,0.003286,44.045295,...,0.103381,15.152211,0.274193,5.659963,0.106079,3.663501,0.056445,5.260544,0.393322,39.582541
GK_y,-0.003325,1.953064,-0.000335,-0.161194,-1.314435,-320.834053,-1.219228,-127.413961,-0.096511,44.045295,...,0.043205,15.152211,-0.159968,5.659963,0.045835,3.663501,0.017375,5.260544,-0.122052,39.582541
tpr_x,0.158708,1.953064,-0.095327,-0.161194,30.049989,-320.834053,28.985077,-127.413961,1.663383,44.045295,...,0.450927,15.152211,1.594376,5.659963,0.589439,3.663501,0.026908,5.260544,2.123198,39.582541
tpr_y,-0.143996,1.953064,0.168197,-0.161194,-7.555009,-320.834053,-9.248325,-127.413961,-0.025295,44.045295,...,-0.875565,15.152211,-1.051392,5.659963,-0.542715,3.663501,-0.172737,5.260544,-1.461647,39.582541
Aggression_x,-0.011321,1.953064,0.018617,-0.161194,-2.443401,-320.834053,-1.923439,-127.413961,-0.192154,44.045295,...,0.004682,15.152211,-0.311966,5.659963,-0.047460,3.663501,-0.019287,5.260544,-0.320319,39.582541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Penalty Taking_y,0.005772,1.953064,-0.017088,-0.161194,-1.239233,-320.834053,-1.208320,-127.413961,-0.059476,44.045295,...,0.001608,15.152211,-0.183338,5.659963,0.026525,3.663501,0.008237,5.260544,-0.087012,39.582541
Tackling_x,-0.006855,1.953064,0.016657,-0.161194,2.256528,-320.834053,1.851654,-127.413961,0.222593,44.045295,...,0.013090,15.152211,0.020927,5.659963,-0.002159,3.663501,-0.031868,5.260544,-0.096101,39.582541
Tackling_y,0.014955,1.953064,0.002472,-0.161194,-0.385583,-320.834053,-0.485933,-127.413961,0.020857,44.045295,...,0.129059,15.152211,-0.054933,5.659963,0.126194,3.663501,0.055055,5.260544,0.118888,39.582541
Technique_x,0.048792,1.953064,0.012479,-0.161194,9.461073,-320.834053,8.985398,-127.413961,0.576825,44.045295,...,0.147349,15.152211,0.989121,5.659963,0.216445,3.663501,0.010891,5.260544,1.229727,39.582541


In [62]:
with pd.ExcelWriter(f"{CURRENT_DATAFRAME_NAME}_Standardized_coefficients.xlsx") as writer:
    formula_df[formula_df.index.str.contains("_x")].to_excel(writer, index=True, sheet_name="x")
    formula_df[formula_df.index.str.contains("_y")].to_excel(writer, index=True, sheet_name="y")

In [63]:
attribute = "PassDead"

In [64]:
custom_tpr_df = pd.DataFrame({
 'tpr_x': 78,
 'pas_x': 90,
 'hed_x': 66,
 'dri_x': 68,
 'tpr_y': 79,
 'def_y': 63}.values(), columns=["custom"], index=reg_tpr_cols)

NameError: name 'reg_tpr_cols' is not defined

In [ ]:
custom_tpr_df[f"{attribute}_coef"] = formula_df[f"{attribute}_coef"].tolist()
custom_tpr_df[f"{attribute}_intercept"] = formula_df[f"{attribute}_intercept"].tolist()
custom_tpr_df["coef_result"] = custom_tpr_df[f"{attribute}_coef"] * custom_tpr_df["custom"]
custom_tpr_df["formula_result"] = custom_tpr_df["coef_result"].sum() + custom_tpr_df[f"{attribute}_intercept"].iloc[0]
custom_tpr_df["formula_result"] = round(custom_tpr_df["formula_result"].iloc[0])

In [ ]:
custom_tpr_df

In [ ]:
formula_df[["GCA_coef","GCA_intercept"]]

In [ ]:
merged_df.iloc[155]